In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.datasets import make_regression, make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler


import mlflow
from mlflow.models import infer_signature
from mlflow import MlflowClient
from mlflow.server import get_app_client
from mlflow.server.auth.client import AuthServiceClient

# from bulkboto3 import BulkBoto3
from dotenv import load_dotenv
load_dotenv()

import yaml
import boto3
import pickle
import warnings
import datetime as dt
warnings.filterwarnings('ignore')

os.environ['AWS_ACCESS_KEY_ID'] = "admin" # логин от s3
os.environ['AWS_SECRET_ACCESS_KEY'] = "23wesdxc" # пароль от s3
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://localhost:9000" # адрес s3
os.environ['MLFLOW_TRACKING_USERNAME'] = "admin" # логин MLFlow
os.environ['MLFLOW_TRACKING_PASSWORD'] = "23wesdxc" # пароль MLFlow 

# Подключение к MLFlow
mlflow.set_tracking_uri(uri=os.getenv("MLFLOW_TRACKING_URI"))

# # Подключение к s3
# session = boto3.session.Session()
# s3 = session.client(
#     service_name='s3',
#     endpoint_url=os.environ['MLFLOW_S3_ENDPOINT_URL']
# )

config = yaml.safe_load(open("./config.yaml"))

## Администрирование MLFlow

In [3]:
client = AuthServiceClient(os.getenv("MLFLOW_TRACKING_URI"))

In [11]:
client.create_experiment_permission(
    experiment_id="1",
    username="qwer",
    permission="EDIT"
)

In [ ]:
# Создание нового пользователя
client.create_user("newuser", "newpassword")

# Удаление пользователя (возможно только после удаления всех доступов)
client.delete_user("newuser")

# Обновление пароля
client.update_user_password("admin", "testpass")

# Создать привелегии
client.create_experiment_permission(
    experiment_id="1",
    username="newuser",
    permission="EDIT" # READ / MANAGE https://mlflow.org/docs/latest/auth/index.html#how-it-works
)

# Обновить привелегии
client.update_experiment_permission(
    experiment_id="1",
    username="newuser",
    permission="MANAGE"
)

# Отозвать привелегии
client.delete_experiment_permission(
    experiment_id="1",
    username="newuser"
)

# Админские права
client.update_user_admin(
    username="newuser",
    is_admin=True
)

In [5]:
user = client.get_user("newuser")
print(f"user_id: {user.id}")
print(f"username: {user.username}")
print(f"password_hash: {user.password_hash}")
print(f"is_admin: {user.is_admin}")

user_id: 4
username: newuser
password_hash: REDACTED
is_admin: False


In [6]:
ep = client.get_experiment_permission("1", "newuser")
print(f"experiment_id: {ep.experiment_id}")
print(f"user_id: {ep.user_id}")
print(f"permission: {ep.permission}")

RestException: RESOURCE_DOES_NOT_EXIST: Experiment permission with experiment_id=1 and username=newuser not found